To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [1]:
! pip install langdetect

In [2]:
import gensim

In [3]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt

from langdetect import detect
import re

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt

In [69]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	 chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n','') for i in Corpus]
    return corpus


"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus,Substr):
    clear_corpus = []
    for text in Corpus:
        indexes = []
        text_len = len(text)
        try:
            for i in range(0,text_len):
                res = re.search(Substr,text[i])
                if res != None:
                    indexes.append(i)
                
            #delete garbage word from text
            for index in indexes:
                del text[index]
        
            clear_corpus.append(text)
        except:
            clear_corpus.append("error")
        
    return clear_corpus


"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

RAWDATA 

In [6]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

Downloading: 100%|██████████| 23450/23450 [00:16<00:00, 1336.46rows/s]


(23450, 4)

In [70]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [8]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


DATA PREPROC

In [9]:
#transform text to lower case
corpus = SupportRawDataframe.description
corpus.astype('str')

corpus = lowerCase(corpus)

#getting language for text corpus
corpus_lang = getTextLanguage(corpus)

In [10]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [71]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
5,chat transcript:visitor: здраствуйтеana: здрав...,chat,ps,0,ru
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en


In [12]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag,lang
23440,"chat transcript:visitor: hello, i bought pubg ...",chat,other,1,en
23441,chat transcript:visitor: good dayi accidentall...,facebook,other,1,en
23442,chat transcript:visitor: payment not going thr...,chat,other,1,en
23443,chat transcript:visitor: paid for faceit subsc...,chat,other,1,en
23444,"chat transcript:visitor: hello, i am trying to...",chat,other,1,en
23445,"chat transcript:visitor: hi, i made a pruchase...",chat,other,1,en
23446,"chat transcript:visitor: hi, how long will it ...",chat,other,1,en
23447,chat transcript:visitor: i bought playerunknow...,chat,other,1,en
23448,chat transcript:visitor: good day i took the w...,chat,other,1,en
23449,chat transcript:visitor: hivisitor: hellovisit...,chat,other,1,en


In [72]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,chat transcript:visitor: i want to buy with pa...,chat,ps,0,en
1,chat transcript:visitor: hell i had a problem ...,chat,ps,0,en
2,chat transcript:visitor: لا استطيع الشراء ومعل...,chat,ps,0,en
3,chat transcript:visitor: im having trouble wit...,chat,ps,0,en
4,chat transcript:visitor: hiana: hello. how can...,chat,ps,0,en
6,chat transcript:visitor: hellovisitor: why i c...,chat,ps,0,en
7,chat transcript:visitor: not letting me buy an...,chat,ps,0,en
8,chat transcript:visitor: مرحباana: hello!visit...,chat,ps,0,en
9,chat transcript:visitor: hi im trying to buy v...,chat,ps,0,en
11,"chat transcript:visitor: hi, i tried purchasin...",chat,ps,0,en


In [73]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [74]:
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[chat, transcript:visitor:, i, want, to, buy, ...",chat,ps,0,en
1,"[chat, transcript:visitor:, hell, i, had, a, p...",chat,ps,0,en
2,"[chat, transcript:visitor:, لا, استطيع, الشراء...",chat,ps,0,en
3,"[chat, transcript:visitor:, im, having, troubl...",chat,ps,0,en
4,"[chat, transcript:visitor:, hiana:, hello., ho...",chat,ps,0,en
6,"[chat, transcript:visitor:, hellovisitor:, why...",chat,ps,0,en
7,"[chat, transcript:visitor:, not, letting, me, ...",chat,ps,0,en
8,"[chat, transcript:visitor:, مرحباana:, hello!v...",chat,ps,0,en
9,"[chat, transcript:visitor:, hi, im, trying, to...",chat,ps,0,en
11,"[chat, transcript:visitor:, hi,, i, tried, pur...",chat,ps,0,en


In [75]:
texts = list(SupportDataframe_eng.description)


In [76]:
#cleaning text after garbage
tests_clear=re.sub(r'[;/)(?!\.:,""«»\s]', ' ', str(tests_clear))# убираем пунктуацию

In [85]:
Trash=['i', 'had', 'hey', 'chat', 'hi', 'hello', 'transcript', 'visitor' ]#список мусорных слов

In [86]:
for item in Trash:
    tests_clear = clearTextAfterGarbage(texts, item)

In [89]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\phomi\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [90]:
from nltk.corpus import stopwords #убираем стоп-слова
tests_clear = [word for word in tests_clear if word not in stopwords.words('english')]

In [91]:
SupportDataframe_eng['description'] = tests_clear
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,"[want, to, buy, and, show, me, a, problem, cal...",chat,ps,0,en
1,"[hell, a, problem, yesterday, about, people, b...",chat,ps,0,en
2,"[لا, استطيع, الشراء, ومعلومات, الشراء, صحيحة, ...",chat,ps,0,en
3,"[with, xsolla, auth, 4, digit, code, a, pymaen...",chat,ps,0,en
4,"[how, can, help, pay, with, i, don't, credit, ...",chat,ps,0,en
6,"[why, cant, pay, ana:, hello., can, i, frome, ...",chat,ps,0,en
7,"[not, me, buy, please, a, moment, while, issue...",chat,ps,0,en
8,"[مرحباana:, have, problem, try, purchase, with...",chat,ps,0,en
9,"[buy, v, bucks, but, it, me, pay, using, my, g...",chat,ps,0,en
11,"[robux, now, but, i, could, fix, that, me?ana:...",chat,ps,0,en


TRAIN TEST SPLIT DATAFRAMES

In [92]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [114]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

In [109]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5)

In [110]:
skf.get_n_splits(descriptions, categories)


5

In [111]:
StratifiedKFold(n_splits=2, random_state=None, shuffle=False)
for train_index, test_index in skf.split(descriptions, categories):
    print("TRAIN:", train_index, "TEST:", test_index)
    XTrain, XTest = descriptions[train_index], descriptions[test_index]
    YTrain, YTest = categories[train_index], categories[test_index]

TRAIN: [  636   637   638 ... 16010 16011 16012] TEST: [   0    1    2 ... 5742 5743 5744]
TRAIN: [    0     1     2 ... 16010 16011 16012] TEST: [ 636  637  638 ... 8309 8310 8311]
TRAIN: [    0     1     2 ... 16010 16011 16012] TEST: [ 1272  1273  1274 ... 10876 10877 10878]
TRAIN: [    0     1     2 ... 16010 16011 16012] TEST: [ 1907  1908  1909 ... 13443 13444 13445]
TRAIN: [    0     1     2 ... 13443 13444 13445] TEST: [ 2542  2543  2544 ... 16010 16011 16012]


FROM TEXTS TO VECTORS

In [115]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(XTrain)

In [116]:
Imdb_w2v.train(XTrain,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(4190915, 6402035)

In [117]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [118]:
TrainVecs

array([[ 0.1865325 ,  0.46442073, -0.23852889, ..., -0.41430715,
         0.16443613,  0.23875056],
       [-0.33260119,  0.53136696, -0.15285421, ..., -0.50617694,
        -0.43151004, -0.07658123],
       [-0.14100243,  0.245011  , -0.36080362, ..., -0.01373388,
        -0.65528355, -0.15640588],
       ...,
       [ 0.03487626,  0.68989208, -0.27268282, ..., -0.26796157,
         0.41204825,  0.27301141],
       [-0.29031105,  0.44219781,  0.03807305, ..., -0.35096483,
        -0.31535108, -0.24749881],
       [ 0.07848376,  0.39434554,  0.09196175, ..., -0.58371463,
         0.02969594, -0.47073112]])

In [119]:
Imdb_w2v.train(XTest, total_examples = Imdb_w2v.corpus_count, epochs = Imdb_w2v.epochs)

(1044127, 1602085)

In [120]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

In [121]:
TestVecs

array([[-0.26728292, -0.03236846,  0.2486817 , ..., -0.64846413,
        -0.39156702, -0.40661497],
       [-0.19541903,  0.17584781,  0.15554323, ..., -0.65159086,
        -0.33481602, -0.10125418],
       [-0.43666372,  0.20149899,  0.06992521, ..., -0.42392199,
        -0.48976124, -0.36117264],
       ...,
       [ 0.11728496, -0.00844389,  0.26055553, ..., -0.54148058,
         0.19388261, -0.40351709],
       [-0.47834229,  0.05328676,  0.56830452, ..., -0.58952636,
        -0.46193742, -0.1045236 ],
       [-0.34521712,  0.34511149,  0.06555718, ..., -0.53661617,
         0.30322676,  0.51735684]])

CROSSVALIDATION AND BUILD CLASSIFIER

In [122]:
#старая кросс-валидация, не выполнять
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 1000, random_state = 40)

In [123]:
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = 5)
print(Scores)
print(np.mean(Scores))

[0.88919235 0.89894655 0.89383294 0.88988676 0.88129637]
0.8906309948658754


In [124]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_iter=1000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=40, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [125]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 410  225]
 [ 330 2238]]
              precision    recall  f1-score   support

           0       0.55      0.65      0.60       635
           1       0.91      0.87      0.89      2568

    accuracy                           0.83      3203
   macro avg       0.73      0.76      0.74      3203
weighted avg       0.84      0.83      0.83      3203

